In [1]:
import matplotlib.pyplot as plt
import numpy as np


#X1 transaction date	
# X2 house age	
# X3 distance to the nearest MRT station	
# X4 number of convenience stores	
# X5 latitude	
# X6 longitude	Y house price of unit area


In [97]:
data = np.genfromtxt("real_estate_valuation.txt")
X = np.array(data[:-20,1:6])
y = np.array(data[:-20,6:7])

X_test = np.array(data[-20:,1:6])
y_test = np.array(data[-20:,6:7])
# print(y)
# y.shape
# data[:,0]
# print(y.shape)
regr = linear_model.LinearRegression()

# Train the model using the training sets
regr.fit(X, y)


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [102]:
predict = regr.predict(X_test)
print("Coefficients:\n", regr.coef_)
print("Mean Squared Error: %2f" % mean_squared_error(y_test, predict))
print("Variance Score: %.2f" % r2_score(y_test, predict))


Coefficients:
 [[ 1.19777065e-03 -2.50278748e-05 -1.08375280e-05 -2.21192519e-04
  -1.15513425e-01]]
Mean Squared Error: 0.000126
Variance Score: 0.49
